In [1]:
# notebook to evaluate the information retrieval results from an external system (i.e. not pyterrier)
import pyterrier as pt
if not pt.started():
    pt.init()


PyTerrier 0.9.2 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:

import xml.etree.ElementTree as ET
import pandas as pd

def load_topics(path):
    with open(path) as f:
        root = ET.fromstring(f.read())
    topic_dict = {}
    for topic in root.findall("topic"):
        topic_id = topic.findtext("id")
        topic_query = topic.findtext("query")
        if topic_id and topic_query:
            topic_dict[topic_id] = topic_query.strip().lower()
    topics = pd.DataFrame(topic_dict.items(), columns=["qid", "query"]) 
    topics["query"] = topics["query"].str.replace(r'\W+', ' ', regex=True)
    return topics

In [4]:
topics = load_topics("../dataset/topics/topics.txt")
qrels = pt.io.read_qrels("../dataset/assessments/qrels.txt") # type: ignore
qcred = pt.io.read_qrels("../dataset/assessments/qcredibility.txt") # type: ignore
qread = pt.io.read_qrels("../dataset/assessments/qreadability.txt") # type: ignore

all_qs = [("qrels", qrels), ("qcred", qcred), ("qread", qread)]

In [5]:
all_passages = pd.read_csv("../dataset/Webdoc/data/txt_min_length_50.tsv", sep="\t")

In [6]:
qrels_in_passages = qrels.merge(all_passages, left_on="docno", right_on="docid", how="inner")
# count the number of documents per query and return new dataframe
number_if_documents_per_query = qrels_in_passages.groupby("qid", as_index=False).count()[["qid", "docno"]]

In [7]:
import pyterrier as pt

def run_experiment(pipeline, simple_name, topics, qrels, eval_metrics=["map", "bpref", "ndcg_cut_10"]):
    experiments = []
    for name, q in qrels:
        # change pipeline name to include the name of the query

        exp = pt.Experiment([pipeline], topics, q, eval_metrics, names=[name + '_' + simple_name], perquery=False)
        experiments.append(exp)
    return pd.concat(experiments, axis=0)

In [8]:
# load all ranking per query files in rankings/simple_name/
import glob
import os
# list all subdirectories in rankings/
ranking_candidates = os.listdir("rankings")
# make sure only directories are considered
ranking_candidates = [candidate for candidate in ranking_candidates if os.path.isdir(f"rankings/{candidate}")]


all_results = []
for path in glob.glob(f"rankings/{simple_name}/*.csv"):
    results = pd.read_csv(path)
    all_results.append(results)
all_results = pd.concat(all_results, axis=0)

['colbert-pubmed-bert-rankings.csv', 'colbertv2.0', 'colbert-pubmed-bert-over_50-rankings.csv', 'colbertv2.0-over_50-rankings.csv', 'colbert-pubmed-bert-rankings_at_10.csv', 'colbert-pubmed-bert', 'colbert-pubmed-bert-rankings_at_10-paras_over_50.csv', 'rankings.csv']


NameError: name 'simple_name' is not defined

In [ ]:

results = run_experiment(all_results,simple_name, topics, all_qs, ["map", "bpref", "ndcg_cut_10"])
results.to_csv("results/" + simple_name + "_results.csv")

In [ ]:
results

,name,map,bpref,ndcg_cut_10
0,qrels_colbert-v2-per-query,0.330816,0.346085,0.649604
0,qcred_colbert-v2-per-query,0.473128,0.466647,0.634022
0,qread_colbert-v2-per-query,0.407758,0.379656,0.799140


In [ ]:
# visualize ndcg@10 for each query
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
# increase plot size
plt.rcParams['figure.figsize'] = [25, 10]
sns.set_style("ticks")

results = pd.read_csv("results/" + simple_name + "_results.csv")
# results = results[results["measure"] == "ndcg_cut_10"]

results["qid"] = results["qid"].astype(str)
# add number of documents per query to qid
results = results.merge(number_if_documents_per_query, on="qid")

results = results.sort_values(by="docno", ascending=True)

results['qid_label'] = results['qid'] + "\n(" + results['docno'].astype(str) + ")"
# plot all ndcg@10 values for each query and for each score in  barplots
# add x-axis label
p = sns.barplot(x="qid_label", y="value", hue="name", data=results)
p.set(xlabel='Query ID (number of evaluated documents)', ylabel='ndcg@10')

KeyError: 'qid'

In [ ]:
results.head()

,name,map,bpref,ndcg_cut_10
0,qrels_colbert-pubmed-bert-rankings_at_10-paras...,0.003569,0.007963,0.079914
0,qcred_colbert-pubmed-bert-rankings_at_10-paras...,0.003599,0.007787,0.094078
0,qread_colbert-pubmed-bert-rankings_at_10-paras...,0.003266,0.006953,0.122413
